# Central Limit Theorem & Sampling Distribution

In this notebook, we examine the Central Limit Theorem (CLT) and sampling distribution using simulation. 

### Central Limit Theorem

Q: Suppose that you gather $n=10$ independent observations, and find that the average is $1.9$. Suppose that one of your friend gathers another bacth of $10$ independent observations, will the average of the new sample be larger or small than $1.9$?

> Refresher:  The **Central Limit Theorem** states that, given a sufficiently large sample size, the distribution of the sample mean of independent, identically distributed random variables approaches a normal (Gaussian) distribution, regardless of the original population's distribution, provided the population has a finite mean and variance.



Using rigorous notation, the basic Central Limit Theorem states the following. 

Let $X_1, X_2, \dots, X_n$ be a random sample of size $n$ drawn from a population with mean $\mu$ and finite variance $\sigma^2$. Define the sample mean as:

$$
\bar{X}_n = \frac{1}{n} \sum_{i=1}^n X_i
$$

Then, as $n \to \infty$, the distribution of the standardized sample mean approaches the standard normal distribution:

$$
\frac{\bar{X}_n - \mu}{\sigma / \sqrt{n}} \xrightarrow{d} \mathcal{N}(0, 1)
$$

That is, for large $n$, the sample mean $\bar{X}_n$ is approximately normally distributed, regardless of the original distribution of the $X_i$’s.


In [4]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from scipy import stats
import ipywidgets as widgets
from pathlib import Path

random_seed=42
sns.set_theme()
rng = np.random.default_rng(random_seed)

In [5]:
def sample_mean_dist(n=5, reps=5000):
    data = rng.exponential(scale=1, size=(reps, n))
    means = data.mean(axis=1)
    plt.figure(figsize=(6,4))
    sns.histplot(means, bins=30, stat='density', label='Sample means')
    plt.axvline(1, ls='--', label='True mean')
    # Overlay normal density curve
    x = np.linspace(means.min(), means.max(), 200)
    normal_density = stats.norm.pdf(x, loc=1, scale=1/np.sqrt(n))
    plt.plot(x, normal_density, color='red', label='Normal (CLT)')
    plt.title(f"Sampling distribution (n={n})")
    plt.legend()
    plt.show()

widgets.interact(sample_mean_dist, n=[2,5,10,30,50,100]);


interactive(children=(Dropdown(description='n', index=1, options=(2, 5, 10, 30, 50, 100), value=5), IntSlider(…

### Sampling Distribution

In the previous example, we know that the sample mean $\bar{X}_n$ follows a normal distribution asymptotically
$$\frac{\bar{X}_n - \mu}{\sigma / \sqrt{n}} \xrightarrow{d} \mathcal{N}(0, 1).$$ 
If we consider a simple linear regression, $i=1,2,\ldots, n$
$$y_i = \beta_1 x_i + \beta_0 \epsilon_i.$$
We can show that the OLS estimator for $\beta_1$ is also in the form of an average
$$
\hat{\beta}_1 = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2}.
$$

In the numeric example below, we will generate data from a model 
$$y = 2x + \epsilon,$$ 
where $x \sim \mathcal{N}(0, 2)$ and $\epsilon$ can be selected from uniform $U(-5,5)$, normal $\mathcal{N}(0, 2)$, or Poisson with mean $4$ (the constant mean can be handled by the intercept).

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression

# Set random generator
rng = np.random.default_rng()

def sample_slope_dist(n=5, eps_type='normal', reps=5000):
    slopes = []

    for _ in range(reps):
        x = rng.normal(loc=0, scale=np.sqrt(2), size=n).reshape(-1, 1)

        if eps_type == 'uniform':
            epsilon = rng.uniform(low=-5, high=5, size=n)
        elif eps_type == 'normal':
            epsilon = rng.normal(loc=0, scale=np.sqrt(2), size=n)
        elif eps_type == 'poisson':
            epsilon = rng.poisson(lam=4, size=n)
        else:
            raise ValueError("eps_type must be 'uniform', 'normal', or 'poisson'")
        
        y = 2 * x.flatten() + epsilon
        model = LinearRegression().fit(x, y)
        slopes.append(model.coef_[0])

    # Plot
    plt.figure(figsize=(6, 4))
    sns.histplot(slopes, bins=30, stat='density', color='skyblue', label='Estimated slopes')
    plt.axvline(2, ls='--', color='black', label='True slope = 2')

    # Normal approximation overlay (optional, assumes CLT)
    mean = np.mean(slopes)
    std = np.std(slopes)
    x_vals = np.linspace(min(slopes), max(slopes), 200)
    normal_density = stats.norm.pdf(x_vals, loc=mean, scale=std)
    plt.plot(x_vals, normal_density, color='red', label='Normal Approximation', linewidth=2)

    plt.title(f"Sampling Distribution of $\\hat{{\\beta}}_1$ (n={n}, $\\epsilon$={eps_type})")
    plt.xlabel("Estimated slope ($\\hat{\\beta}_1$)")
    plt.ylabel("Density")
    plt.legend()
    plt.show()

# Interactivity
widgets.interact(
    sample_slope_dist,
    n=[2, 5, 10, 30, 50, 100],
    eps_type=['uniform', 'normal', 'poisson']
);


interactive(children=(Dropdown(description='n', index=1, options=(2, 5, 10, 30, 50, 100), value=5), Dropdown(d…